## Vincent tool for preparing templates and atlases

In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns

# importing private libs and functions (imaging_arsenal version, need to import the parent dir as well)
private_lib_path = Path('/scratch/imaging_arsenal')
sys.path.append(str(private_lib_path))
sys.path.append(str(private_lib_path / 'utils_'))
print('private lib ready:',(private_lib_path / 'utils_' /'utils_private.py').exists())
from utils_.utils_private import report_cpu
report_cpu()

/opt/conda/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


private lib ready: True
Total CPUs: 30
Total Memory: 186.37 GB
Available Memory: 180.28 GB


### Transform atlas

In [46]:
## helper functions
def resample_atlas_to_target_space(atlas_name, target_template_dir='/scratch/atlas/templateflow/tpl-MNI152NLin2009aAsym', s_atlas_dir = '/scratch/atlas/AtlasPack-docker', t_atlas_dir = '/scratch/atlas/AtlasPack-Vincent', s_space="MNI152NLin6Asym", t_space="MNI152NLin2009cAsym"):
    """
    Resample an atlas from MNI152NLin6Asym space to MNI152NLin2009cAsym space.
    
    Parameters:
    - atlas_name: Name of the atlas to be resampled.
    - target_template_dir: Path to the target template directory in MNI152NLin2009cAsym space.
    - transformation_matrix_path: Path to the transformation matrix (.h5).
    - s_atlas_dir: Directory containing source atlas images.
    - t_atlas_dir: Directory where transformed atlas images will be saved.
    - s_space: Source space (default is MNI152NLin6Asym).
    - t_space: Target space (default is MNI152NLin2009cAsym).
    """
    import ants
    import nibabel as nib
    import json
    from nipype.interfaces import utility as niu
    from nipype.pipeline import engine as pe
    import os
    import shutil

    # Step 1: Load the atlas image
    s_atlas_path = Path(s_atlas_dir)
    t_atlas_path = Path(t_atlas_dir)
    t_atlas_path.mkdir(parents=True, exist_ok=True)
    s_atlas_img_path = s_atlas_path / atlas_name / f"{atlas_name}_space-{s_space}_res-01_dseg.nii.gz"
    t_atlas_img_path = t_atlas_path / atlas_name / f"{atlas_name}_space-{t_space}_res-01_dseg.nii.gz"
    os.makedirs(t_atlas_img_path.parent, exist_ok=True)
    #
    target_template_path = Path(target_template_dir) 
    target_template_img_path = target_template_path / f"tpl-{t_space}_res-01_desc-brain_T1w.nii.gz"
    transformation_matrix_path = target_template_path / ("tpl-"+t_space+"_from-"+s_space+"_mode-image_xfm.h5")
    #tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5

    ## Transformation
    transformed_atlas_img = apply_transform_with_ants(s_atlas_img_path, transformation_matrix_path, target_template_img_path, t_atlas_img_path)
    # Save the transformed atlas image
    transformed_atlas_img.to_filename(t_atlas_img_path)

    print(f"Transformed atlas saved to {t_atlas_img_path}")
    # Step 6: Create BIDS-compatible JSON metadata file
    metadata = {"Name": "4S Atlas (transformed by vincent)",
                "Description": "Resampled from MNI152NLin6Asym space to MNI152NLin2009cAsym space using TemplateFlow transform by Vincent.",
                "SourceSpace": "MNI152NLin6Asym",
                "TargetSpace": "MNI152NLin2009cAsym",
                "TemplateFlowTemplateTargetT1": str(target_template_img_path),
                "TemplateFlowTransform": str(transformation_matrix_path),
                "Interpolation": "GenericLabel"
            }

    # Save the BIDS JSON metadata file
    bids_json_path = t_atlas_path / atlas_name / f"{atlas_name}_space-{t_space}_res-01_dseg.json"
    with open(bids_json_path, 'w') as json_file:
        json.dump(metadata, json_file, indent=4)
    print(f"Atlas resampled and saved to {t_atlas_img_path}. BIDS JSON metadata saved to {bids_json_path}.")
    s_label_file_path = s_atlas_path / atlas_name / f"{atlas_name}_dseg.tsv"
    t_label_file_path = t_atlas_path / atlas_name / f"{atlas_name}_dseg.tsv"
    # Copy the label file directly (assuming labels remain the same)
    
    shutil.copyfile(s_label_file_path, t_label_file_path)
    print(f"Label file copied to {t_label_file_path}.")

    ### Transformation surface files need to be done separately (not covered here yet)

    return [transformed_atlas_img, metadata]

def apply_transform_with_ants(input_img_path, transform_path, reference_image_paht, output_img_path):
    """
    Apply the transformation from MNI152NLin6Asym to MNI152NLin2009cAsym to the given atlas using ANTsPy.
    
    Parameters:
    - input_img_path: Path to the atlas image in MNI152NLin6Asym space
    - transform_path: Path to the transformation matrix to apply (from TemplateFlow)
    - reference_image_path: The target image in MNI152NLin2009cAsym space (template)
    - output_img_path: Path where the transformed image will be saved
    """
    import ants
    # Load input image and target reference image (MNI152NLin2009cAsym)
    input_img = ants.image_read(str(input_img_path))
    print("input image loaded:", input_img.shape)
    ref_img = ants.image_read(str(reference_image_paht))
    print("reference image loaded:", ref_img.shape)
    # Load the transformation matrix
    transform_matrix = ants.read_transform(str(transform_path))
    print("transform loaded:", transform_matrix)
    # Apply the transformation
    transformed_atlas = ants.apply_transforms(fixed=ref_img, moving=input_img, transformlist=[str(transform_path)], interpolator='nearestNeighbor')
    print(transformed_atlas)
    return transformed_atlas

In [48]:
## Source atlas
import nibabel as nib

source_alas_dir = Path('/scratch/atlas/AtlasPack-docker')
atlas_name_list = ["atlas-4S156Parcels", "atlas-4S256Parcels", "atlas-4S356Parcels", "atlas-4S456Parcels", "atlas-4S556Parcels",
                   "atlas-4S656Parcels", "atlas-4S756Parcels", "atlas-4S856Parcels", "atlas-4S956Parcels", "atlas-4S1056Parcels"]
source_space = "MNI152NLin6Asym"

atlas_name = "atlas-4S256Parcels"
source_atlas_file = source_alas_dir / atlas_name / f"{atlas_name}_space-{source_space}_res-01_dseg.nii.gz"

## Target atlas 
target_altas_dir = Path('/scratch/atlas/AtlasPack-Vincent')
target_space = "MNI152NLin2009cAsym"
target_atlas_file = target_altas_dir / atlas_name / f"{atlas_name}_space-{target_space}_res-01_dseg.nii.gz"

## transform
#transformed_atlas_img = resample_to_img(source_atlas_img, nib.load(target_atlas_file), interpolation='nearest')

## save transformed atlas
#os.makedirs(target_atlas_file.parent, exist_ok=True)
#nib.save(transformed_atlas_img, target_atlas_file)
#print(f"Saved transformed atlas to: {target_atlas_file}")

for atlas_name in atlas_name_list:
    [transformed_atlas_img, metadata] = resample_atlas_to_target_space(atlas_name, target_template_dir='/scratch/atlas/templateflow/tpl-MNI152NLin2009aAsym', 
                                                                       s_atlas_dir = '/scratch/atlas/AtlasPack-docker', t_atlas_dir = '/scratch/atlas/AtlasPack-Vincent', 
                                                                       s_space="MNI152NLin6Asym", t_space="MNI152NLin2009cAsym") 

input image loaded: (182, 218, 182)
reference image loaded: (193, 229, 193)
transform loaded: ANTsTransform
	 Type       : CompositeTransform
	 Dimension  : 3
	 Precision  : float

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (193, 229, 193)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (96.0, 132.0, -78.0)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

Transformed atlas saved to /scratch/atlas/AtlasPack-Vincent/atlas-4S156Parcels/atlas-4S156Parcels_space-MNI152NLin2009cAsym_res-01_dseg.nii.gz
Atlas resampled and saved to /scratch/atlas/AtlasPack-Vincent/atlas-4S156Parcels/atlas-4S156Parcels_space-MNI152NLin2009cAsym_res-01_dseg.nii.gz. BIDS JSON metadata saved to /scratch/atlas/AtlasPack-Vincent/atlas-4S156Parcels/atlas-4S156Parcels_space-MNI152NLin2009cAsym_res-01_dseg.json.
Label file copied to /scratch/atlas/AtlasPack-Vincent/atlas-4S156Parcels/atlas-4S156Parcels_dseg.tsv.
input image loaded: (182, 218, 182)
reference image loaded: (193, 229

### 1. update templateflow
Now using datalad, not statble in China.

In [ ]:
import templateflow as tfl
import os
from pathlib import Path
import nibabel as nib
from nilearn.image import resample_to_img

# Where to cache templates (change if you want a different folder)
tf_folder = "/scratch/templateflow"
os.environ.setdefault('TF_CACHE_DIR', tf_folder)
print(f"TemplateFlow cache set to: {os.environ['TF_CACHE_DIR']}")

TemplateFlow cache set to: /scratch/templateflow


confirm atlas

### 2. Transform the 